In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

In [ ]:
import pandas     as pd
import numpy      as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

In [ ]:
#get html and scrap with pandas
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
#rename columns to proper Names
data.rename(index=str,columns={0:'Postcode',1:'Borough',2:'Neighbourhood'},inplace = True)
#first row is not valid
data.drop(['0'],inplace = True)
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
data.drop(data[data['Borough'] == 'Not assigned'].index,inplace = True)
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
data['Neighbourhood'][data['Neighbourhood'] == 'Not assigned'] = data['Borough'][data['Neighbourhood'] == 'Not assigned']

In [ ]:
#reset Index I think this is not mandatory
data.reset_index(inplace = True)
data.drop(columns=['index'], axis = 1,inplace = True)

In [ ]:
data['Neighbourhood'] = '%' + data['Neighbourhood'].astype(str) + '%'
data = data.groupby(['Postcode' ,'Borough' ],as_index = False)['Neighbourhood'].sum()
data['Neighbourhood'] = data['Neighbourhood'].str.replace('%%',', ')
data['Neighbourhood'] = data['Neighbourhood'].str.replace('%','')

In [ ]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.rename(index=str,columns={'Postal Code':'Postcode'},inplace = True)

In [ ]:
data = data.join(geo_data.set_index('Postcode'), on='Postcode')

In [ ]:
address = 'Toronto City, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

I real don't have any new idea about clustering of Toronto city Neighbourhoods and I don't Like to do just the same as we did to New york city, I just decide to cluster them in 5 by their Latitude and Longitude
for a future use of Dorn postmans.

In [ ]:
data_cluster = data.drop(columns={'Postcode','Borough','Neighbourhood'})
K_clusters = 5
kmeans = KMeans(n_clusters=K_clusters, random_state=0).fit(data_cluster)
data['Cluster'] = kmeans.labels_

In [ ]:
map_clusters = folium.Map(width=1000,height=500,location=[latitude, longitude], zoom_start=11)

x = np.arange(K_clusters)
ys = [i + x + (i*x)**2 for i in range(K_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(data['Latitude'],data['Longitude'], data['Neighbourhood'], data['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker( [lat, lon]
                       , radius       =  5
                       , popup        =  label
                       , color        =  rainbow[cluster-1]
                       , fill         =  True
                       , fill_color   =  rainbow[cluster-1]
                       , fill_opacity =  0.7
                       , parse_html   =  False      ).add_to(map_clusters)       
map_clusters